In [ ]:
# import statements

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os 
from skimage.color import rgb2gray
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#functions

# this function reads and image with the directory file_path, resizes the image, and converts it to grayscale
# @param file_path is the directory of the image
def ReadResizeConvert(file_path):
    """reads image, resizes, converts to grayscale"""
    img = cv2.imread(file_path)
    assert type(img) != type(None)
    newImg = cv2.resize(img, (28, 28))
    assert newImg.shape == (28, 28, 3)
    gray_image = cv2.cvtColor(newImg, cv2.COLOR_BGR2GRAY)
    assert gray_image.shape == (28, 28)
    return gray_image

# this function uses directoryName to find the list of images in that directory. 
# It then uses the function ReadResizeConvert to change the images 
# then it appends the images to an array already created outside the function.
# @param directoryName is a string that is part of the directory
def imageToArray(directoryName, localFinalImagesArr, localFinalAnswersArr, answer):
    listOfImages = os.listdir(directoryName)
    directory = directoryName + '/'
    for i in listOfImages:
        nameOfImage = directory + i
        assert type(nameOfImage) == type('')
        output = ReadResizeConvert(nameOfImage)
        localFinalImagesArr.append(output)
        localFinalAnswersArr.append(answer)

In [ ]:
#finalImagesArr is the array in which we put the images from the array data1 that we keep adding to
#finalAnswersArr contains the corresponding labels for the images in myData

finalImagesArrTrain = []
finalAnswersArrTrain = []

imageToArray('/Users/23amrutad/Desktop/cup-original', finalImagesArrTrain, finalAnswersArrTrain, 0)
imageToArray('/Users/23amrutad/Desktop/coke-original', finalImagesArrTrain, finalAnswersArrTrain, 1)

x_test = np.array(finalImagesArrTrain)
y_test = np.array(finalAnswersArrTrain)


finalImagesArrTest = []
finalAnswersArrTest = []

imageToArray('/Users/23amrutad/Desktop/cup_test', finalImagesArrTest, finalAnswersArrTest, 0)
imageToArray('/Users/23amrutad/Desktop/cup-sunny', finalImagesArrTest, finalAnswersArrTest, 0)
imageToArray('/Users/23amrutad/Desktop/coke_test', finalImagesArrTest, finalAnswersArrTest, 1)
imageToArray('/Users/23amrutad/Desktop/coke-sunny', finalImagesArrTest, finalAnswersArrTest, 1)

x_train = np.array(finalImagesArrTest)
y_train = np.array(finalAnswersArrTest)

In [ ]:
#this code makes a list of arrays with entries for x_train and the corresponding y_train
#it changes the list to an np array, shuffles the array, 
#and puts the x_train and y_train, now shuffled, back in their respective training arrays
#we have to shuffle them together because if we shuffle them separately, it goes out of sync

xy_list = [];
count = 0;
for _ in x_train:
    myArr = [x_train[count], y_train[count]]
    xy_list.append(myArr)
    count+=1;
xy_nparr = np.array(xy_list)
np.random.shuffle(xy_nparr)
print (xy_nparr.shape)
count = 0;
for _ in xy_nparr:
    x_train[count] = xy_nparr[count][0]
    y_train[count] = xy_nparr[count][1]
    count += 1;

In [ ]:
print ('shape of x-train is ', x_train.shape)
print ('shape of y-train is ', y_train.shape)
print ('shape of x-test is ', x_test.shape)
print ('shape of y-test is ', y_test.shape)

x_train_short = [];
y_train_short = [];
x_test_short = [];
y_test_short = [];
for i in range(y_train.shape[0]):
  if (y_train[i] == 0 or y_train[i] == 1):
    x_train_short.append(x_train[i])
    y_train_short.append(y_train[i])
  else: 
    pass
for i in range(y_test.shape[0]):
  if (y_test[i] == 0 or y_test[i] == 1):
    x_test_short.append(x_test[i])
    y_test_short.append(y_test[i])
  else: 
    pass

x_train = np.array(x_train_short)
y_train = np.array(y_train_short)
x_test = np.array(x_test_short)
y_test = np.array(y_test_short)

print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
 
 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
 

 
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.softmax))
 
 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=50)
model.evaluate(x_test, y_test)

In [ ]:
model.predict(x_test)